# 02_ocr_testing.ipynb
## Pruebas de Extracción OCR

Objetivo: Prototipar y validar OCR (Landing AI vs DeepSeek)

In [6]:
import sys
from pathlib import Path
# Añadir el directorio PADRE (la raíz del proyecto) al path
sys.path.insert(0, str(Path.cwd().parent))

from src.extractors import OCRExtractor
from src.utils.logger import get_logger
from src.utils.config import EXCEL_IMAGES_DIR, PDF_IMAGES_DIR

logger = get_logger(__name__)

ImportError: cannot import name 'process_all_extracted_text' from 'src.extractors.structure_parser' (c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\src\extractors\structure_parser.py)

## 1. Verificar Imágenes Disponibles

In [7]:
# Listar imágenes disponibles
excel_images = list(EXCEL_IMAGES_DIR.glob("*.png"))
pdf_images = list(PDF_IMAGES_DIR.glob("*.png"))
all_images = excel_images + pdf_images

print(f"Imágenes de Excel: {len(excel_images)}")
print(f"Imágenes de PDF: {len(pdf_images)}")
print(f"Total: {len(all_images)}")

if all_images:
    print(f"\nPrimeras imágenes:")
    for img in all_images[:5]:
        print(f"  - {img.name}")
else:
    print("\n⚠️ No hay imágenes. Ejecuta notebook 01 primero.")

NameError: name 'EXCEL_IMAGES_DIR' is not defined

## 2. Inicializar OCR Extractor

In [ ]:
# Crear extractor
# Cambiar provider en .env: OCR_PROVIDER=landing_ai o deepseek

extractor_landing = OCRExtractor(provider="landing_ai")
print("✅ Landing AI extractor inicializado")

extractor_deepseek = OCRExtractor(provider="deepseek")
print("✅ DeepSeek extractor inicializado")

## 3. Prueba OCR: Landing AI

In [ ]:
# Probar con primera imagen
if excel_images:
    test_image = excel_images[0]
    print(f"Testing Landing AI con: {test_image.name}")
    
    result_landing = extractor_landing.extract_text(str(test_image))
    
    print(f"\nResultado:")
    print(f"  Status: {result_landing.get('status')}")
    print(f"  Provider: {result_landing.get('provider')}")
    
    if result_landing.get('status') == 'success':
        text = result_landing.get('text', '')
        print(f"  Texto extraído (primeros 200 chars):")
        print(f"  {text[:200]}...")
        print(f"  Total chars: {len(text)}")
    else:
        print(f"  ⚠️ Error: {result_landing.get('message')}")

## 4. Prueba OCR: DeepSeek

In [ ]:
# Probar con DeepSeek
if excel_images:
    test_image = excel_images[0]
    print(f"Testing DeepSeek con: {test_image.name}")
    
    result_deepseek = extractor_deepseek.extract_text(str(test_image))
    
    print(f"\nResultado:")
    print(f"  Status: {result_deepseek.get('status')}")
    print(f"  Provider: {result_deepseek.get('provider')}")
    
    if result_deepseek.get('status') == 'success':
        text = result_deepseek.get('text', '')
        print(f"  Texto extraído (primeros 200 chars):")
        print(f"  {text[:200]}...")
        print(f"  Total chars: {len(text)}")
    else:
        print(f"  ⚠️ Error: {result_deepseek.get('message')}")

## 5. Extracción de Estructura

In [ ]:
# Analizar estructura del texto extraído
if excel_images and 'result_landing' in locals():
    extracted_text = result_landing.get('text', '')
    
    structure = extractor_landing.extract_structure(extracted_text)
    
    print("Estructura detectada:")
    print(f"  Líneas: {len(structure.get('lines', []))}")
    print(f"  Tablas detectadas: {len(structure.get('tables', []))}")
    
    key_fields = structure.get('key_fields', {})
    print(f"\n  Campos clave:")
    print(f"    - Fechas: {key_fields.get('dates', [])}")
    print(f"    - Montos: {key_fields.get('amounts', [])}")
    print(f"    - Conceptos: {len(key_fields.get('concepts', []))}")

## 6. Parser: Estructuración JSON

In [ ]:
# Usar StructureParser para mejor formato
parser = StructureParser()

if excel_images and 'result_landing' in locals():
    extracted_text = result_landing.get('text', '')
    
    structured = parser.parse(extracted_text)
    
    print("\nDatos estructurados:")
    print(f"  Status: {structured.get('status', 'N/A')}")
    print(f"  Tablas: {len(structured.get('tables', []))}")
    
    fields = structured.get('fields', {})
    print(f"\n  Campos extraídos:")
    print(f"    - Fechas: {fields.get('dates', [])}")
    print(f"    - Montos: {fields.get('amounts', [])}")
    print(f"    - Cuentas: {fields.get('accounts', [])}")
    
    # Mostrar primera tabla si existe
    tables = structured.get('tables', [])
    if tables:
        print(f"\n  Primera tabla:")
        table = tables[0]
        print(f"    Headers: {table.get('headers', [])}")
        print(f"    Filas: {table.get('row_count', 0)}")

## 7. Comparación: Landing AI vs DeepSeek

In [ ]:
# Comparar calidad de extracción
if 'result_landing' in locals() and 'result_deepseek' in locals():
    text_landing = result_landing.get('text', '')
    text_deepseek = result_deepseek.get('text', '')
    
    print("COMPARACIÓN: Landing AI vs DeepSeek")
    print("="*60)
    print(f"\nLanding AI:")
    print(f"  - Status: {result_landing.get('status')}")
    print(f"  - Caracteres: {len(text_landing)}")
    print(f"  - Líneas: {len(text_landing.split(chr(10)))}")
    
    print(f"\nDeepSeek:")
    print(f"  - Status: {result_deepseek.get('status')}")
    print(f"  - Caracteres: {len(text_deepseek)}")
    print(f"  - Líneas: {len(text_deepseek.split(chr(10)))}")
    
    # Similitud aproximada
    if text_landing and text_deepseek:
        common_chars = len(set(text_landing) & set(text_deepseek))
        similarity = common_chars / max(len(text_landing), len(text_deepseek)) * 100
        print(f"\nSimilitud: {similarity:.1f}%")
    print("="*60)

## 8. Procesar todas las imágenes

In [ ]:
# Procesar lote de imágenes
print("Procesando todas las imágenes con OCR...")
print("="*60)

results_summary = {
    'success': 0,
    'error': 0,
    'total': len(all_images)
}

for idx, image in enumerate(all_images[:5], 1):  # Primeras 5 para testing
    print(f"\n[{idx}/{min(5, len(all_images))}] {image.name}")
    
    result = extractor_landing.extract_text(str(image))
    
    if result.get('status') == 'success':
        text = result.get('text', '')
        print(f"  ✅ Extraído: {len(text)} caracteres")
        results_summary['success'] += 1
    else:
        print(f"  ❌ Error: {result.get('message')}")
        results_summary['error'] += 1

print(f"\n{'='*60}")
print(f"RESUMEN:")
print(f"  Exitosos: {results_summary['success']}")
print(f"  Errores: {results_summary['error']}")
print(f"  Total: {results_summary['total']}")

## 9. Guardar resultados OCR

In [ ]:
# Guardar ejemplo de OCR + estructura en JSON
if 'structured' in locals():
    output_file = Path('ocr_test_output.json')
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(structured, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Resultados guardados en: {output_file}")
    print(f"\nPrimeras líneas:")
    print(json.dumps(structured, ensure_ascii=False, indent=2)[:500])

## 10. Resumen y Conclusiones

In [ ]:
print("="*60)
print("RESUMEN: EXTRACCIÓN OCR")
print("="*60)
print(f"\n✅ Imágenes procesadas: {len(all_images)}")
print(f"\n✅ Providers testeados:")
print(f"   - Landing AI (Enterprise)")
print(f"   - DeepSeek (Económico)")
print(f"\n✅ Estructura detectada:")
print(f"   - Fechas")
print(f"   - Montos (S/, USD)")
print(f"   - Conceptos")
print(f"   - Tablas")
print(f"\n✅ Próximo paso:")
print(f"   → Notebook 03: CLIP Embeddings & Espacio Compartido")
print("="*60)